## Usage

Optimizes a given geometry in Gaussian 16 and scans a user given bond to create a potential energy curve. The output can directly be used as input for DFTB parametrization with the TANGO program.

## Workflow

Use GaussView to draw the structure you want to use for repulsive fitting in TANGO. Before saving the geometry go to Tools-->Atom List...-->Edit-->Reorder-->All Atoms (Except the First) by Bonding. Then save the structure as Z-Matrix. Only the geometry will be read by the code, so don't bother about the header lines and the basis set in this file. Then ask yourself which basis set you want to employ and why you choose this study program... after contemplating the life choices that led you here, visit https://www.basissetexchange.org/ and save the whole basis set into one file. Be careful to include all atoms contained in your system and no more! Now you only need to edit the values in the next code cell to your liking. Run all cells and go drink a coffee (or seventy-three if you have a large system :D)

## User Input:

In [9]:
molecule_name = "h3f"   # label or name of the molecule to be scanned --> Gaussian files for opt and scan will be named accordingly
memory = 25             # amount of RAM in GB allocated for the Gaussian calculations
n_cpus = 6              # number of cpus used for the Gaussian calculations
method = "ump2(full)"   # method employed for the calculations
title_card = "Basis Set from basissetexchange.org:  Zr: cc-pVTZ-PP   F, H: cc-pVTZ" # title card used for the .gjf files, ape likes to insert the source of the used basis set and the exact flavor of basis set
scan_bond = "B4"        # provide the bond label as defined in the geom_file that needs to be scanned
lower_bound = 0.7       # lower bound of the scanned bond as multiple of the equilibrium bond distance
upper_bound = 1.75      # upper bound of the scanned bond as multiple of the equilibrium bond distance
scan_step_number = (upper_bound - lower_bound) * 100 # defines the number of points minus one between the lower and upper bound that are calculated --> you can hardcode it if you like or take a multiple of the given value
geom_file = "geom.gjf"  # path to the .gjf containing the geometry of interest
basisset_file = "basis-set.info" # path to the file containing the gaussian basis set; website for obtaining basis-sets: https://www.basissetexchange.org/

## Check if the user is trolling and the bond to be scanned is non-existing

In [10]:
with open(geom_file, "r") as file:
    lines = file.readlines()
# Locate the line containing the scan_bond string
for i, line in enumerate(lines):
    if line.strip().startswith(f"{scan_bond} "):
        parts = line.split()
        if len(parts) == 2:
            break
else:
    raise ValueError(f"Error: Could not find the line starting with '{scan_bond}'.\n Please provide a valid bond to be scanned.")

## Block that sets the header lines of the .gjf for the geometry optimization

In [11]:
header = [
    f"%chk={molecule_name}.chk\n",
    f"%mem={memory}GB\n",
    f"%nprocshared={n_cpus}\n",
    f"# {method}/gen pseudo=read guess=mix scf=(maxcycle=500, yqc, tight) OPT(maxcycles=1000)\n",     # gen enables the use of custom basis sets; pseudo=read enables the use of pseudo potentials; guess=mix mixes the HOMO and LUMO as initial guess for the HF wavefunction
    "\n",
    f"{title_card}\n",
    "\n"
]

## Definition of the function reading in the geometry

In [12]:
def read_geometry_block(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Skip initial empty lines
    index = 0
    while index < len(lines) and lines[index].strip() == "":
        index += 1

    # Skip the first non-empty line
    if index < len(lines):
        index += 1

    # Skip the next two empty lines
    empty_line_count = 0
    while index < len(lines) and empty_line_count < 2:
        if lines[index].strip() == "":
            empty_line_count += 1
        index += 1

    # Read the geometry block
    geometry_block = []
    empty_line_count = 0
    while index < len(lines) and empty_line_count < 2:
        geometry_block.append(lines[index])
        if lines[index].strip() == "":
            empty_line_count += 1
        index += 1

    # Strip empty lines from the end of the geometry_block
    while geometry_block and geometry_block[-1].strip() == "":
        geometry_block.pop()

    return geometry_block

## Definition of the function reading in the Basis Set

In [13]:
def read_basis_set(file_path):
    with open(file_path, 'r') as file:
        basisset = file.readlines()

    # Remove blank lines at the start
    while basisset and basisset[0].strip() == "":
        basisset.pop(0)

    # Remove blank lines at the end
    while basisset and basisset[-1].strip() == "":
        basisset.pop()

    # Add a blank line at the beginning and the end
    basisset.insert(0, "\n")
    basisset.append("\n")
    
    return basisset

## Extract the Geometry and Basis Set from the input files and write the Gaussian input file

In [14]:
import os

# Read the geometry block and basis set
input_geometry = read_geometry_block(geom_file)
basisset = read_basis_set(basisset_file)

# Concatenate the header, input_geometry and basisset
output_content = header + input_geometry + basisset

# Create the 'opt' directory if it doesn't exist
os.makedirs('opt', exist_ok=True)

# Write the concatenated list to the Gaussian input file in the 'opt' directory
with open(os.path.join('opt', f"{molecule_name}.gjf"), "w") as file:
    for line in output_content:
        file.write(line)

## Run the Gaussian geometry optimization

In [15]:
import os
import subprocess

# Create the bash script content
bash_script_content = f"""#!/bin/bash
module purge
module load gaussian/g16
cd opt
g16 {molecule_name}.gjf
cd ..
"""

# Write the bash script to a file
bash_script_path = "run_gaussian_opt.sh"
with open(bash_script_path, "w") as file:
    file.write(bash_script_content)

# Make the bash script executable
os.chmod(bash_script_path, 0o755)

# Execute the bash script
subprocess.run(["./" + bash_script_path], check=True)

# Delete the bash script after execution
os.remove(bash_script_path)

Removing compiler-rt version 2022.1.0
Use `module list` to view any remaining dependent modules.
Loading gaussian/g16
  Loading requirement: pgi/16.5


KeyboardInterrupt: 

## Check if geometry did converge

In [ ]:
import os

# Path to the log file
opt_log_file_path = os.path.join("opt", f"{molecule_name}.log")

# Check if the last line starts with "Normal termination"
with open(opt_log_file_path, "r") as file:
    lines = file.readlines()
    if lines and lines[-1].strip().startswith("Normal termination"):
        print("Geometry optimization completed successfully.")
    else:
        raise RuntimeError("Error: Geometry optimization didn't terminate properly.")

RuntimeError: Error: Geometry optimization didn't terminate properly.

## Definition of the function extracting the converged geometry

In [ ]:
import re

# Function to extract the optimized structure from the log file
def extract_optimized_structure(opt_log_file_path):
    with open(opt_log_file_path, "r") as file:
        lines = file.readlines()

    # Find the start of the geometry block
    start_index = None
    for i, line in enumerate(lines):
        if "Final structure in terms of initial Z-matrix:" in line:
            start_index = i + 1
            break

    if start_index is None:
        raise RuntimeError("Error: Could not find the start of the geometry block.")

    # Extract the geometry block
    geometry_block = []
    for line in lines[start_index:]:
        if re.match(r'^\d', line.strip()):
            break
        if "Variables:" in line:
            geometry_block.append("\n")
        else:
            # Replace commas with spaces and add spaces around equal signs
            modified_line = line.replace(",", "   ")
            modified_line = re.sub(r'=', '   ', modified_line)
            geometry_block.append(modified_line)

    return geometry_block

## Block that sets the header lines of the .gjf for the bond scan

In [ ]:
header = [
    f"%chk={molecule_name}.chk\n",
    f"%mem={memory}GB\n",
    f"%nprocshared={n_cpus}\n",
    f"# scan {method}/gen pseudo=read guess=mix scf=(maxcycle=500, yqc, tight)\n",     # gen enables the use of custom basis sets; pseudo=read enables the use of pseudo potentials; guess=mix mixes the HOMO and LUMO as initial guess for the HF wavefunction
    "\n",
    f"{title_card}\n",
    "\n"
]

## Extract the optimized geometry from the .log file and write the input file for the bond scan

In [ ]:
# Extract the optimized structure and prepend the charge and spin multiplicity taken from the input_geometry
optimized_structure = extract_optimized_structure(opt_log_file_path)
optimized_structure.insert(0, input_geometry[0])

# Concatenate the header, optimized_structure and basisset
output_content = header + optimized_structure + basisset

# Create the 'scan' directory if it doesn't exist
os.makedirs('scan', exist_ok=True)

# Write the concatenated list to the Gaussian input file in the 'scan' directory
with open(os.path.join('scan', f"{molecule_name}.gjf"), "w") as file:
    for line in output_content:
        file.write(line)

## Insert the bond scan in the Gaussian input file

In [ ]:
import os

# Path to the .gjf file for the scan
scan_file_path = os.path.join("scan", f"{molecule_name}.gjf")

# Function to locate the line, extract the float, calculate new values, and overwrite the line
def update_scan_bond(scan_file_path, scan_bond):
    with open(scan_file_path, "r") as file:
        lines = file.readlines()

    # Locate the line containing the scan_bond string
    for i, line in enumerate(lines):
        if line.strip().startswith(scan_bond):
            # Extract the float value from the line
            parts = line.split()
            if len(parts) == 2:
                equilibrium_dist = float(parts[1])
                minimum_dist = equilibrium_dist * lower_bound
                maximum_dist = equilibrium_dist * upper_bound
                scan_distance = maximum_dist - minimum_dist
                step_size = scan_distance / (scan_step_number)
                # Calculate new values (example calculation)
                new_line = f" {scan_bond}   {minimum_dist:.8f}   {int(scan_step_number)}   {step_size}\n"
                # Overwrite the exact line in the file
                lines[i] = new_line
                break
    else:
        raise ValueError(f"Error: Could not find the line starting with '{scan_bond}'.")

    # Write the updated lines back to the file
    with open(scan_file_path, "w") as file:
        file.writelines(lines)


update_scan_bond(scan_file_path, scan_bond)

## Run the Gaussian bond scan

In [ ]:
import os
import subprocess

# Create the bash script content
bash_script_content = f"""#!/bin/bash
module purge
module load gaussian/g16
cd scan
g16 {molecule_name}.gjf
cd ..
"""

# Write the bash script to a file
bash_script_path = "run_gaussian_scan.sh"
with open(bash_script_path, "w") as file:
    file.write(bash_script_content)

# Make the bash script executable
os.chmod(bash_script_path, 0o755)

# Execute the bash script
subprocess.run(["./" + bash_script_path], check=True)

# Delete the bash script after execution
os.remove(bash_script_path)

## Check if the bond scan terminated properly

In [ ]:
import os

# Path to the log file
scan_log_file_path = os.path.join("scan", f"{molecule_name}.log")

# Check if the last line starts with "Normal termination"
with open(scan_log_file_path, "r") as file:
    lines = file.readlines()
    if lines and lines[-1].strip().startswith("Normal termination"):
        print("Bond scan completed successfully.")
    else:
        raise RuntimeError("Error: Bond scan didn't terminate properly.")

## Extract the geometries of the scan for TANGO to use

In [ ]:
import os

# Periodic table mapping from atomic number to element symbol
periodic_table = {
    1: "H", 2: "He", 3: "Li", 4: "Be", 5: "B", 6: "C", 7: "N", 8: "O", 9: "F", 10: "Ne",
    11: "Na", 12: "Mg", 13: "Al", 14: "Si", 15: "P", 16: "S", 17: "Cl", 18: "Ar", 19: "K", 20: "Ca",
    21: "Sc", 22: "Ti", 23: "V", 24: "Cr", 25: "Mn", 26: "Fe", 27: "Co", 28: "Ni", 29: "Cu", 30: "Zn",
    31: "Ga", 32: "Ge", 33: "As", 34: "Se", 35: "Br", 36: "Kr", 37: "Rb", 38: "Sr", 39: "Y", 40: "Zr",
    41: "Nb", 42: "Mo", 43: "Tc", 44: "Ru", 45: "Rh", 46: "Pd", 47: "Ag", 48: "Cd", 49: "In", 50: "Sn",
    51: "Sb", 52: "Te", 53: "I", 54: "Xe", 55: "Cs", 56: "Ba", 57: "La", 58: "Ce", 59: "Pr", 60: "Nd",
    61: "Pm", 62: "Sm", 63: "Eu", 64: "Gd", 65: "Tb", 66: "Dy", 67: "Ho", 68: "Er", 69: "Tm", 70: "Yb",
    71: "Lu", 72: "Hf", 73: "Ta", 74: "W", 75: "Re", 76: "Os", 77: "Ir", 78: "Pt", 79: "Au", 80: "Hg",
    81: "Tl", 82: "Pb", 83: "Bi", 84: "Po", 85: "At", 86: "Rn", 87: "Fr", 88: "Ra", 89: "Ac", 90: "Th",
    91: "Pa", 92: "U", 93: "Np", 94: "Pu", 95: "Am", 96: "Cm", 97: "Bk", 98: "Cf", 99: "Es", 100: "Fm",
    101: "Md", 102: "No", 103: "Lr", 104: "Rf", 105: "Db", 106: "Sg", 107: "Bh", 108: "Hs", 109: "Mt",
    110: "Ds", 111: "Rg", 112: "Cn", 113: "Nh", 114: "Fl", 115: "Mc", 116: "Lv", 117: "Ts", 118: "Og"
}

# Function to extract the required data from the log file
def extract_data_from_log(scan_log_file_path, output_file_path):
    with open(scan_log_file_path, "r") as file:
        lines = file.readlines()

    all_blocks = []
    for i, line in enumerate(lines):
        if "Standard orientation:" in line:
            start_index = i + 5 
            block_data = []
            while start_index < len(lines) and not lines[start_index].strip().startswith('-'):
                parts = lines[start_index].split()
                if len(parts) >= 6:
                    atomic_number = int(parts[1])
                    element = periodic_table.get(atomic_number, "X")
                    x = parts[3]
                    y = parts[4]
                    z = parts[5]
                    block_data.append(f"{element} {x} {y} {z}")
                start_index += 1
            if block_data:
                all_blocks.append(f"{len(block_data)}\n\n" + "\n".join(block_data))

    with open(output_file_path, "w") as file:
        for block in all_blocks:
            file.write(block + "\n")

scan_log_file_path = os.path.join("scan", f"{molecule_name}.log")
output_file_path = f"traj_{molecule_name}.xyz"

extract_data_from_log(scan_log_file_path, output_file_path)

## Extract the energies of the scan for TANGO to use

In [ ]:
import re

hartree_to_eV = 27.211407953

# Function to extract energies from the log file
def extract_energies(scan_log_file_path):
    with open(scan_log_file_path, "r") as file:
        lines = file.readlines()

    energies = []
    for line in lines:
        if "EUMP2" in line:
            match = re.search(r"EUMP2\s*=\s*([-\d\.D]+[+-]\d+)", line)
            if match:
                energy = match.group(1).replace('D', 'E')
                try:
                    energy_eV = float(energy) * hartree_to_eV
                    energies.append(energy_eV)
                except ValueError as e:
                    print(f"Error converting energy: {energy} - {e}")
    return energies

# Function to write energies and distances to output file
def write_energies_to_file(energies, minimum_dist, step_size, scan_step_number, output_file_path):
    distances = [minimum_dist + i * step_size for i in range(scan_step_number + 1)]
    with open(output_file_path, "w") as file:
        for distance, energy in zip(distances, energies):
            file.write(f"{distance} {energy}\n")

# Extract energies and write to file
energy_output_file_path = f"energy_{molecule_name}.dat"
energies = extract_energies(scan_log_file_path)
write_energies_to_file(energies, minimum_dist, step_size, scan_step_number, energy_output_file_path)